In [1]:
#Loading Libraries
import numpy as np 
import pandas as pd
import numpy as np
import os
import imutils
import dlib 
import cv2 
import imageio
from imutils import face_utils

In [2]:
def rect_to_bb(rect):
    # take a bounding predicted by dlib and convert it
    # to the format (x, y, w, h) as we would normally do
    # with OpenCV
    x = rect.left()
    y = rect.top()
    w = rect.right() - x
    h = rect.bottom() - y

    # return a tuple of (x, y, w, h)
    return (x, y, w, h)

def shape_to_np(shape, dtype="int"):
    # initialize the list of (x, y)-coordinates
    coords = np.zeros((68, 2), dtype=dtype)

    # loop over the 68 facial landmarks and convert them
    # to a 2-tuple of (x, y)-coordinates
    for i in range(0, 68):
    	coords[i] = (shape.part(i).x, shape.part(i).y)

    # return the list of (x, y)-coordinates
    return coords

rect_to_bb: Function to convert a bounding box predicted by dlib to (x, y, w, h) format as used in OpenCV

rect: dlib rectangle objec

shape_to_np: Function to convert facial landmark coordinates from dlib's shape predictor to NumPy array format
shape: dlib shape object, representing facial landmarks# dtype: data type of the output NumPy array, default is 'int't



In [3]:
def crop_and_save_image(img, img_path, write_img_path, img_name):
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor('./MIRACL-VC1_all_in_one/shape_predictor_68_face_landmarks.dat')
    # load the input image, resize it, and convert it to grayscale

    image = cv2.imread(img_path)
    image = imutils.resize(image, width=500)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # detect faces in the grayscale image
    rects = detector(gray, 1)
    if len(rects) > 1:
    	print( "ERROR: more than one face detected")
    	return
    if len(rects) < 1:
    	print( "ERROR: no faces detected")
    	return

    for (i, rect) in enumerate(rects):
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        name, i, j = 'mouth', 48, 68
        clone = gray.copy()

        (x, y, w, h) = cv2.boundingRect(np.array([shape[i:j]]))        
        roi = gray[y:y+h, x:x+w]
        roi = imutils.resize(roi, width = 250, inter=cv2.INTER_CUBIC)        
        print('cropped/' + write_img_path)
        imageio.imwrite('cropped/' + write_img_path, roi)


In [4]:
os.listdir('./MIRACL-VC1_all_in_one')
predictor = dlib.shape_predictor('./MIRACL-VC1_all_in_one/shape_predictor_68_face_landmarks.dat')

In [5]:

people = ['F01','F02','F04','F05','F06','F07','F08','F09', 'F10','F11','M01','M02','M04','M07','M08']
data_types = ['phrases']
folder_enum = ['01','02','03','04','05','06','07','08','09','10']
instances = ['01','02','03','04','05','06','07','08','09','10']

words = ['Stop navigation', 'Excuse me', 'I am sorry', 'Thank you', 'Good bye', 'I love this game', 'Nice to meet you', 'You are welcome', 'How are you', 'Have a good time']          
words_di = {i:words[i] for i in range(len(words))}

In [6]:
#crearting new folder for cropped images
#if not os.path.exists('cropped'):
    #os.mkdir('cropped')

In [7]:
import shutil

def crop_one_person():      
    
#    people = ['F01','F02','F04','F05','F06','F07','F08','F09', 'F10','F11','M01','M02','M04','M07','M08']
#    data_types = ['words']
#   folder_enum = ['01','02']
#    instances = ['01','02']

    i = 1
    for person_ID in people:
        if not os.path.exists('cropped/' + person_ID ):
            os.mkdir('cropped/' + person_ID + '/')

        for data_type in data_types:
            if not os.path.exists('cropped/' + person_ID + '/' + data_type):
                os.mkdir('cropped/' + person_ID + '/' + data_type)

            for phrase_ID in folder_enum:
                if not os.path.exists('cropped/' + person_ID + '/' + data_type + '/' + phrase_ID):
                    # F01/phrases/01
                    os.mkdir('cropped/' + person_ID + '/' + data_type + '/' + phrase_ID)

                for instance_ID in instances:
                    # F01/phrases/01/01
                    directory = './MIRACL-VC1_all_in_one/' + person_ID + '/' + data_type + '/' + phrase_ID + '/' + instance_ID + '/'
                    dir_temp = person_ID + '/' + data_type + '/' + phrase_ID + '/' + instance_ID + '/'
                    print(directory)
                    filelist = os.listdir(directory)
                    if not os.path.exists('cropped/' + person_ID + '/' + data_type + '/' + phrase_ID + '/' + instance_ID):
                        os.mkdir('cropped/' + person_ID + '/' + data_type + '/' + phrase_ID + '/' + instance_ID)

                        for img_name in filelist:
                            if img_name.startswith('color'):
                                image = imageio.imread(directory + '' + img_name)
                                crop_and_save_image(image, directory + '' + img_name,
                                                    dir_temp + '' + img_name, img_name)

    print(f'Iteration : {i}')
    i += 1
  #  shutil.rmtree('cropped')

In [8]:
import time
os.mkdir('cropped')
times = 0
for _ in range(7):
    t1 = time.time()
    crop_one_person()
    t2 = time.time()
    times += (t2 - t1)

print("Average time over 7 iterations : ", times/7)

./MIRACL-VC1_all_in_one/F01/phrases/01/01/


C:\Users\satvi\AppData\Local\Temp\ipykernel_14836\1483237364.py:35: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(directory + '' + img_name)


cropped/F01/phrases/01/01/color_001.jpg
cropped/F01/phrases/01/01/color_002.jpg
cropped/F01/phrases/01/01/color_003.jpg
cropped/F01/phrases/01/01/color_004.jpg
cropped/F01/phrases/01/01/color_005.jpg


KeyboardInterrupt: 

In [ ]:
os.listdir('./cropped')